In [29]:
from uuid import uuid4
from pprint import PrettyPrinter
from copy import deepcopy
import networkx as nx
import plotly.graph_objects as go

def first_true(iterable, default=False, pred=None):
    """Returns the first true value in the iterable.

    If no true value is found, returns *default*

    If *pred* is not None, returns the first item
    for which pred(item) is true.

    """
    # first_true([a,b,c], x) --> a or b or c or x
    # first_true([a,b], x, f) --> a if f(a) else b if f(b) else x
    return next(filter(pred, iterable), default)

pprinter = PrettyPrinter(indent=2)
pprint = lambda t: pprinter.pprint(t)

class State(object):
    def __init__(self,props={}):
        super(State,self).__init__()
        self.id = str(uuid4())
        self.props = props

    def clone(self):
        return State(deepcopy(self.props))
    
    def __eq__(self,other):
        for key in self.props.keys():
            if not self.props[key] in other.props.values():
                return False
        for key in other.props.keys():
            if not other.props[key] in self.props.values():
                return False
        return True
    
    def __str__(self):
        return pprinter.pformat(self.props)
    
    def __repr__(self):
        return self.__str__()

class Action(object):
    def __init__(self):
        super(Action,self).__init__()

    def check_state(self,state):
        return state != None
    
    def transition(self,state):
        return state.clone()

class A0Action(Action):
    """
    "name":"Move core to W",
    "pre":[
        part_with_conditions("initial-core", [
            is_part("core"),
            has_property("pans", 0),
            has_property("gaskets", 0),
            part_location("C")
        ])
    ],
    "post":[
        part_with_conditions("initial-core", [
            part_location("W")
        ])
    ],
    """
    def __init__(self):
        super(A0Action,self).__init__()

    def check_state(self,state):
        passes = False
        cores_at_w = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["pans"] == 0 and \
                state.props[state_key]["gaskets"] == 0 and \
                state.props[state_key]["location"] == "C":
                passes = True
            elif state.props[state_key]["type"] == "core" and \
                state.props[state_key]["location"] == "W":
                cores_at_w += 1
        return passes and cores_at_w == 0
    
    def transition(self,state):
        new_state = state.clone()
        moved = False
        for state_key in new_state.props:
            if not moved and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["pans"] == 0 and \
                new_state.props[state_key]["gaskets"] == 0 and \
                new_state.props[state_key]["location"] == "C":
                new_state.props[state_key]["location"] = "W"
                moved = True
        return new_state
    
class B0Action(Action):
    """
    "name":"Apply tape to pan",
    "pre":[
        part_with_conditions("initial-pan", [
            is_part("pan"),
            has_property("taped", False),
            part_location("P")
        ])
    ],
    "post":[
        part_with_conditions("initial-pan", [
            has_property("taped", True)
        ])
    ],
    """
    def __init__(self):
        super(B0Action,self).__init__()

    def check_state(self,state):
        passes = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["taped"] == False:
                passes = True
        return passes
    
    def transition(self,state):
        new_state = state.clone()
        taped = False
        for state_key in new_state.props:
            if not taped and new_state.props[state_key]["type"] == "pan" and \
                new_state.props[state_key]["taped"] == False:
                new_state.props[state_key]["taped"] = True
                taped = True
        return new_state
    
class B1Action(Action):
    """
    "name":"Caulk Pan",
    "pre":[
        part_with_conditions("taped-pan", [
            is_part("pan"),
            has_property("taped", True),
            has_property("caulked", False),
            part_location("P")
        ])
    ],
    "post":[
        part_with_conditions("taped-pan", [
            has_property("caulked", True)
        ])],
    """
    def __init__(self):
        super(B1Action,self).__init__()

    def check_state(self,state):
        passes = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["taped"] == True and \
                state.props[state_key]["caulked"] == False:
                passes = True
        return passes
    
    def transition(self,state):
        new_state = state.clone()
        caulked = False
        for state_key in new_state.props:
            if not caulked and new_state.props[state_key]["type"] == "pan" and \
                new_state.props[state_key]["taped"] == True and \
                new_state.props[state_key]["caulked"] == False:
                new_state.props[state_key]["caulked"] = True
                caulked = True
        return new_state

class C0Action(Action):
    """
    "name":"Move Pan to W",
    "pre":[
        part_with_conditions("caulked-pan", [
            is_part("pan"),
            has_property("taped", True),
            has_property("caulked", True),
            part_location("P")
        ])
    ],
    "post":[
        part_with_conditions("taped-pan", [
            part_location("W"),
        ])
    ],
    """
    def __init__(self):
        super(C0Action,self).__init__()

    def check_state(self,state):
        passes = False
        pans_at_w = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["taped"] == True and \
                state.props[state_key]["caulked"] == True and \
                state.props[state_key]["location"] != "W":
                passes = True
            elif state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["location"] == "W":
                pans_at_w += 1
        return passes and pans_at_w <= 1
    
    def transition(self,state):
        new_state = state.clone()
        moved = False
        for state_key in new_state.props:
            if not moved and new_state.props[state_key]["type"] == "pan" and \
                new_state.props[state_key]["taped"] == True and \
                new_state.props[state_key]["caulked"] == True and \
                new_state.props[state_key]["location"] != "W":
                new_state.props[state_key]["location"] = "W"
                moved = True
        return new_state

class C1Action(Action):
    """
    "name":"Apply Pan 1 to Core",
    "pre":[
        part_with_conditions("caulked-pan-input", [
            is_part("pan"),
            has_property("caulked",True),
            part_location("W")
        ]),
        part_with_conditions("panned-core", [
            is_part("core"),
            part_location("W"),
            has_property_ge("pans", 0),
            has_property_le("pans", 1)
        ])
    ],
    "post":[
        part_with_conditions("panned-core", [
            has_property("pans",property_add(previous_property("panned-core", "pans"), 1))
        ]),
        destroyed_part("caulked-pan-input")
    ],
    """
    def __init__(self):
        super(C1Action,self).__init__()

    def check_state(self,state):
        passes_pan = False
        passes_core = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["taped"] == True and \
                state.props[state_key]["caulked"] == True and \
                state.props[state_key]["location"] == "W":
                passes_pan = True
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["pans"] <= 1 and \
                state.props[state_key]["pans"] >= 0 and \
                state.props[state_key]["location"] == "W":
                passes_core = True
        return passes_pan and passes_core
    
    def transition(self,state):
        new_state = state.clone()
        deleted = False
        added = False
        state_keys = list(new_state.props.keys())
        for state_key in state_keys:
            if not deleted and new_state.props[state_key]["type"] == "pan" and \
                new_state.props[state_key]["taped"] == True and \
                new_state.props[state_key]["caulked"] == True and \
                new_state.props[state_key]["location"] == "W":
                del new_state.props[state_key]
                deleted = True
            if not added and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["pans"] <= 1 and \
                new_state.props[state_key]["pans"] >= 0 and \
                new_state.props[state_key]["location"] == "W":
                new_state.props[state_key]["pans"] += 1
                added = True
        return new_state
    
class C2Action(Action):
    """
    "name":"Cleanup Core",
    "pre":[
        part_with_conditions("panned-core", [
            is_part("core"),
            part_location("W"),
            has_property("pans", 2),
            has_property("cleaned", False)
        ])
    ],
    "post":[
        part_with_conditions("panned-core", [
            has_property("cleaned", True)
        ])],
    """
    def __init__(self):
        super(C2Action,self).__init__()

    def check_state(self,state):
        passes = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["pans"] == 2 and \
                state.props[state_key]["cleaned"] == False and \
                state.props[state_key]["location"] == "W":
                passes = True
        return passes
    
    def transition(self,state):
        new_state = state.clone()
        cleaned = False
        for state_key in new_state.props:
            if not cleaned and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["pans"] == 2 and \
                new_state.props[state_key]["cleaned"] == False and \
                new_state.props[state_key]["location"] == "W":
                new_state.props[state_key]["cleaned"] = True
                cleaned = True
        return new_state
    
class D0Action(Action):
    """
    "name":"Move Gasket to T",
    "pre":[
        part_with_conditions("default-gasket", [
            is_part("gasket"),
            has_property("caulked", False),
            part_location("G")
        ])
    ],
    "post":[
        part_with_conditions("default-gasket", [
            any_of([
                part_location("T1"),
                part_location("T2"),
                part_location("T3"),
                part_location("T4"),
            ])
        ])
    ],
    """
    def __init__(self):
        super(D0Action,self).__init__()

    def check_state(self,state):
        passes = False
        gaskets_at_t = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["caulked"] == False and \
                state.props[state_key]["location"] != "T":
                passes = True
            elif state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["location"] == "T":
                gaskets_at_t += 1
        return passes and gaskets_at_t <= 3
    
    def transition(self,state):
        new_state = state.clone()
        moved = False
        for state_key in new_state.props:
            if not moved and new_state.props[state_key]["type"] == "gasket" and \
                new_state.props[state_key]["caulked"] == False and \
                new_state.props[state_key]["location"] != "T":
                new_state.props[state_key]["location"] = "T"
                moved = True
        return new_state
    
class D1Action(Action):
    """
    "name":"Caulk Gasket",
    "pre":[
        part_with_conditions("caulked-gasket", [
            is_part("gasket"),
            has_property("caulked", False),
            any_of([
                part_location("T1"),
                part_location("T2"),
                part_location("T3"),
                part_location("T4"),
            ])
        ])
    ],
    "post":[
        
        part_with_conditions("caulked-gasket", [
            has_property("caulked", True),
        ])
    ],
    """
    def __init__(self):
        super(D1Action,self).__init__()

    def check_state(self,state):
        passes = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["caulked"] == False and \
                state.props[state_key]["location"] == "T":
                passes = True
        return passes
    
    def transition(self,state):
        new_state = state.clone()
        caulked = False
        for state_key in new_state.props:
            if not caulked and new_state.props[state_key]["type"] == "gasket" and \
                new_state.props[state_key]["caulked"] == False and \
                new_state.props[state_key]["location"] == "T":
                new_state.props[state_key]["caulked"] = True
                caulked = True
        return new_state
    
class E0Action(Action):
    """
    "name":"Move Gasket to W",
    "pre":[
        part_with_conditions("caulked-gasket", [
            is_part("gasket"),
            has_property("caulked", True),
            any_of([
                part_location("T1"),
                part_location("T2"),
                part_location("T3"),
                part_location("T4"),
            ])
        ])
    ],
    "post":[
        part_with_conditions("caulked-gasket", [
            part_location("W")
        ])
    """
    def __init__(self):
        super(E0Action,self).__init__()

    def check_state(self,state):
        passes = False
        gaskets_at_w = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["caulked"] == True and \
                state.props[state_key]["location"] != "W":
                passes = True
            elif state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["location"] == "W":
                gaskets_at_w += 1
        return passes and gaskets_at_w == 0
    
    def transition(self,state):
        new_state = state.clone()
        moved = False
        for state_key in new_state.props:
            if not moved and new_state.props[state_key]["type"] == "gasket" and \
                new_state.props[state_key]["caulked"] == True and \
                new_state.props[state_key]["location"] != "W":
                new_state.props[state_key]["location"] = "W"
                moved = True
        return new_state
    
class E1Action(Action):
    """
    "name":"Apply Gasket to Core",
    "pre": [
        part_with_conditions("default-gasket", [
            is_part("gasket"),
            has_property("caulked", True),
            part_location("W")
        ]),
        part_with_conditions("cleaned-core", [
            is_part("core"),
            has_property("cleaned", True),
            has_property("pans", 2),
            has_property_ge("gaskets",0),
            has_property_le("gaskets",3),
            has_property("rivets", property_multiply(current_property("cleaned-core", "gaskets"), 4)),
            part_location("W")
        ]),
    ],
    "post": [
        part_with_conditions("cleaned-core", [
            has_property("gaskets", property_add(previous_property("cleaned-core", "gaskets"), 1)),
        ]),
        destroyed_part("default_gasket")
    ],
    """
    def __init__(self):
        super(E1Action,self).__init__()

    def check_state(self,state):
        passes_pan = False
        passes_core = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["caulked"] == True and \
                state.props[state_key]["location"] == "W":
                passes_pan = True
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["cleaned"] == True and \
                state.props[state_key]["gaskets"] <= 3 and \
                state.props[state_key]["rivets"] == state.props[state_key]["gaskets"]*4 and \
                state.props[state_key]["location"] == "W":
                passes_core = True
        return passes_pan and passes_core
    
    def transition(self,state):
        new_state = state.clone()
        deleted = False
        added = False
        state_keys = list(new_state.props.keys())
        for state_key in state_keys:
            if not deleted and new_state.props[state_key]["type"] == "gasket" and \
                new_state.props[state_key]["caulked"] == True and \
                new_state.props[state_key]["location"] == "W":
                del new_state.props[state_key]
                deleted = True
            if not added and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["cleaned"] == True and \
                new_state.props[state_key]["gaskets"] <= 3 and \
                new_state.props[state_key]["rivets"] == new_state.props[state_key]["gaskets"]*4 and \
                new_state.props[state_key]["location"] == "W":
                new_state.props[state_key]["gaskets"] += 1
                added = True
        return new_state
    
class E2Action(Action):
    """
    "name":"Apply Pop-Rivets to Core",
    "pre":[
        part_with_conditions("gasketed-core", [
            is_part("core"),
            has_property("pans", 2),
            has_property("cleaned", True),
            has_property_ge("gaskets",0),
            has_property_le("gaskets",4),
            has_property("rivets", property_multiply(property_subtract(current_property("gasketed-core", "gaskets"),1), 4)),
            part_location("W")
        ])
    ],
    "post":[
        part_with_conditions("cleaned-core", [
            has_property("gaskets", property_add(previous_property("gasketed-core", "gaskets"), 4)),
        ]),
    ],
    """
    def __init__(self):
        super(E2Action,self).__init__()

    def check_state(self,state):
        passes_pan = False
        passes_core = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["cleaned"] == True and \
                state.props[state_key]["gaskets"] <= 4 and \
                state.props[state_key]["rivets"] == (state.props[state_key]["gaskets"]-1)*4 and \
                state.props[state_key]["location"] == "W":
                print("found satisfying core")
                passes_core = True
        return passes_pan and passes_core
    
    def transition(self,state):
        new_state = state.clone()
        added = False
        for state_key in new_state.props:
            if not added and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["cleaned"] == True and \
                new_state.props[state_key]["gaskets"] <= 4 and \
                new_state.props[state_key]["rivets"] == (new_state.props[state_key]["gaskets"]-1)*4 and \
                new_state.props[state_key]["location"] == "W":
                print("incrementing rivets")
                new_state.props[state_key]["rivets"] += 4
                added = True
        return new_state
    
class SpawnCoreAction(Action):
    """
    
    """
    def __init__(self):
        super(SpawnCoreAction,self).__init__()

    def check_state(self,state):
        cores_at_c = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["location"] == "C":
                cores_at_c += 1
        return cores_at_c == 0
    
    def transition(self,state):
        new_state = state.clone()
        new_state.props[str(uuid4())] = {"type":"core","pans":0,"gaskets":0,"rivets":0,"cleaned":0,"location":"C"}
        return new_state
    
class SpawnPanAction(Action):
    """
    
    """
    def __init__(self):
        super(SpawnPanAction,self).__init__()

    def check_state(self,state):
        pans_at_p = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "pan" and \
                state.props[state_key]["location"] == "P":
                pans_at_p += 1
        return pans_at_p == 0
    
    def transition(self,state):
        new_state = state.clone()
        new_state.props[str(uuid4())] = {"type":"pan","taped":False,"caulked":False,"location":"P"}
        return new_state
    
class SpawnGasketAction(Action):
    """
    
    """
    def __init__(self):
        super(SpawnGasketAction,self).__init__()

    def check_state(self,state):
        pans_at_p = 0
        for state_key in state.props:
            if state.props[state_key]["type"] == "gasket" and \
                state.props[state_key]["location"] == "G":
                pans_at_p += 1
        return pans_at_p == 0
    
    def transition(self,state):
        new_state = state.clone()
        new_state.props[str(uuid4())] = {"type":"gasket","caulked":False,"location":"G"}
        return new_state
    
class CompleteAction(Action):
    """
    "name":"Complete Core",
    "pre": [
        part_with_conditions("cleaned-core", [
            is_part("core"),
            has_property("cleaned", True),
            has_property("pans", 2),
            has_property("gaskets",4),
            has_property("rivets", 16),
            part_location("W")
        ]),
    ],
    "post": [
        destroyed_part("cleaned-core")
    ],
    """
    def __init__(self):
        super(CompleteAction,self).__init__()

    def check_state(self,state):
        passes_core = False
        for state_key in state.props:
            if state.props[state_key]["type"] == "core" and \
                state.props[state_key]["cleaned"] == True and \
                state.props[state_key]["gaskets"] == 4 and \
                state.props[state_key]["rivets"] == 16 and \
                state.props[state_key]["location"] == "W":
                passes_core = True
                print("PASSES CompleteAction")
                
        return passes_core
    
    def transition(self,state):
        new_state = state.clone()
        deleted = False
        state_keys = list(new_state.props.keys())
        for state_key in state_keys:
            if not deleted and new_state.props[state_key]["type"] == "core" and \
                new_state.props[state_key]["cleaned"] == True and \
                new_state.props[state_key]["gaskets"] == 4 and \
                new_state.props[state_key]["rivets"] == 16 and \
                new_state.props[state_key]["location"] == "W":
                del new_state.props[state_key]
                deleted = True
        return new_state
    


In [30]:
s = State()

actions = [
    A0Action(),
    B0Action(),
    B1Action(),
    C0Action(),
    C1Action(),
    C2Action(),
    D0Action(),
    D1Action(),
    E0Action(),
    E1Action(),
    E2Action(),
    SpawnCoreAction(),
    SpawnGasketAction(),
    SpawnPanAction(),
    CompleteAction()
]

added = True
states = [State()]
transitions = []

while added:
    # print("State Count: {}".format(len(states)))
    added_states = []
    for state in states:
        for action in actions:
            if action.check_state(state):
                current_states = states+added_states
                # print(current_states)
                new_state = action.transition(state)
                existing = first_true(current_states,default=None,pred=lambda x: x==new_state)
                # print(existing)
                if existing:
                    # print('found existing',existing.id)
                    transitions.append({"parent":state.id,"child":existing.id,"type":type(action).__name__})
                else: 
                    # print('created new',new_state.id)
                    added_states.append(new_state)
                    transitions.append({"parent":state.id,"child":new_state.id,"type":type(action).__name__})
                    
                            
    # print(len(added_states))
    added = len(added_states) > 0
    states = states+added_states

print("States",len(states))
print("Transitions",len(transitions))




States 1536
Transitions 111104


In [20]:
graph = nx.DiGraph()
for state in states:
    graph.add_node(state.id,**state.props)
for transition in transitions:
    graph.add_edge(transition["parent"],transition["child"],label=transition["type"])

pos = nx.spring_layout(graph, dim=3, seed=779)

node_x = []
node_y = []
node_z = []
node_text = []

for node in graph.nodes():
    x, y, z = pos[node]
    node_x.append(x)
    node_y.append(y)
    node_z.append(z)

node_trace = go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers',
    hoverinfo='text',
    name="States",
    marker=dict(
        # showscale=True,
        # colorscale options
        #'Greys' | 'YlGnBu' | 'Greens' | 'YlOrRd' | 'Bluered' | 'RdBu' |
        #'Reds' | 'Blues' | 'Picnic' | 'Rainbow' | 'Portland' | 'Jet' |
        #'Hot' | 'Blackbody' | 'Earth' | 'Electric' | 'Viridis' |
        # colorscale='YlGnBu',
        # reversescale=True,
        # color=[],
        size=5,
        # colorbar=dict(
        #     thickness=15,
        #     title='Node Connections',
        #     xanchor='left',
        #     titleside='right'
        # ),
        line_width=2)
)

edge_trace_info = {}
for actiontype in [type(action).__name__ for action in actions]:
    edge_trace_info[actiontype] = {'label':actiontype,"x":[],"y":[],"z":[]}

for edge in graph.edges():
    label = graph.edges[edge[0],edge[1]]["label"]
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_trace_info[label]["x"].append(x0)
    edge_trace_info[label]["x"].append(x1)
    edge_trace_info[label]["x"].append(None)
    edge_trace_info[label]["y"].append(y0)
    edge_trace_info[label]["y"].append(y1)
    edge_trace_info[label]["y"].append(None)
    edge_trace_info[label]["z"].append(z0)
    edge_trace_info[label]["z"].append(z1)
    edge_trace_info[label]["z"].append(None)

traces = [node_trace]

for edge_trace in edge_trace_info:
    info = edge_trace_info[edge_trace]
    edge_trace = go.Scatter3d(
        x=info["x"], y=info["y"], z=info["z"],
        line=dict(width=1),
        mode='lines',
        name=info["label"]
    )
    traces.append(edge_trace)

fig = go.Figure(data=traces,
             layout=go.Layout(
                title='<br>Network graph made with Python',
                titlefont_size=16,
                showlegend=True,
                hovermode='closest',
                margin=dict(b=20,l=5,r=5,t=40),
                annotations=[ dict(
                    text="Caption Here",
                    showarrow=False,
                    xref="paper", yref="paper",
                    x=0.005, y=-0.002 ) ],
                # xaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                # yaxis=dict(showgrid=False, zeroline=False, showticklabels=False),
                # zaxis=dict(showgrid=False, zeroline=False, showticklabels=False)
                )
                )
fig.show()

In [21]:
fig.write_html("Swarm.html")

In [22]:
actions

In [28]:
print(len([transition for transition in transitions if transition["type"]=="E2Action"]))

0
